In [ ]:
!pip install transformers
!pip install torch
!pip install wordfreq

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from wordfreq import zipf_frequency
import re

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
model.eval()

Sinonimų ir A1-A2 žodžių sarašas:

In [ ]:
synonym_dict = {}
with open("/content/sinonimai_pataisytas.txt", "r") as file:
    for line in file:
        parts = line.strip().split(";")
        word = parts[0].strip()
        synonyms = parts[1:]
        synonym_dict[word] = synonyms
common_words_df = pd.read_excel("/content/dazniausi_zodziai.xlsx")
common_words_list = common_words_df["word"].tolist()

Sudėtingas sakinys:

In [ ]:
list_texts = [
    'Išmoka gimus vienu metu daugiau kaip vienam vaikui skiriama. Ji mokama vadovaujantis Lietuvos Respublikos išmokų vaikams įstatymu.'
]


In [ ]:
def get_bert_candidates(input_text, synonym_dict, tokenizer, model, numb_predictions_displayed=10):
    list_candidates = []
    for word in input_text.split():
        if word not in common_words_list:
            if word in synonym_dict:
                synonyms = synonym_dict[word]
                list_candidates.append((word, synonyms))
        else:
            replace_word_mask = input_text.replace(word, '[MASK]')
            text = f'[CLS]{replace_word_mask} [SEP] {input_text} [SEP] '
            tokenized_text = tokenizer.tokenize(text)
            masked_index = [i for i, x in enumerate(tokenized_text) if x == '[MASK]'][0]
            indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
            segments_ids = [0] * len(tokenized_text)
            tokens_tensor = torch.tensor([indexed_tokens])
            segments_tensors = torch.tensor([segments_ids])

            with torch.no_grad():
                outputs = model(tokens_tensor, token_type_ids=segments_tensors)
                predictions = outputs[0][0][masked_index]
            predicted_ids = torch.argsort(predictions, descending=True)[:numb_predictions_displayed]
            predicted_tokens = tokenizer.convert_ids_to_tokens(list(predicted_ids))
            list_candidates.append((word, predicted_tokens))
    return list_candidates


Atsakymas:

In [ ]:
for input_text in list_texts:
    new_text = input_text
    for word in input_text.split():
        if word not in common_words_list:
            bert_candidates = get_bert_candidates(word, synonym_dict, tokenizer, model)
            print("Kandidatai", word, ":", bert_candidates)  # Kandidatai
            for word_to_replace, l_candidates in bert_candidates:
                print("Keistinas žodis:", word_to_replace)  # Pakeičiamas žodis
                print("Kandidatų sarašas:", l_candidates)  # Kandidatų sarašas

                tuples_word_zipf = sorted(l_candidates, key=lambda x: zipf_frequency(x, 'lt'), reverse=True)
                print("Atrinkti kandidatai:", tuples_word_zipf)  # Atrinkti kandidatai
                new_text = re.sub(word_to_replace, tuples_word_zipf[0], new_text)
    print("Originalus sakinys: ", input_text)
    print("Supaprastintas sakinys:", new_text, "\n")

Kandidatai Išmoka : []
Kandidatai gimus : []
Kandidatai vaikui : [('vaikui', ['kūdikiui'])]
Keistinas žodis: vaikui
Kandidatų sarašas: ['kūdikiui']
Atrinkti kandidatai: ['kūdikiui']
Kandidatai skiriama. : []
Kandidatai Ji : []
Kandidatai mokama : [('mokama', ['duodama'])]
Keistinas žodis: mokama
Kandidatų sarašas: ['duodama']
Atrinkti kandidatai: ['duodama']
Kandidatai vadovaujantis : [('vadovaujantis', ['vadovausiantis', 'besivadovaujantis', 'vadovaudavęs', 'remiantis'])]
Keistinas žodis: vadovaujantis
Kandidatų sarašas: ['vadovausiantis', 'besivadovaujantis', 'vadovaudavęs', 'remiantis']
Atrinkti kandidatai: ['remiantis', 'vadovausiantis', 'besivadovaujantis', 'vadovaudavęs']
Kandidatai Respublikos : []
Kandidatai išmokų : [('išmokų', ['kompensacijų', 'pašalpų'])]
Keistinas žodis: išmokų
Kandidatų sarašas: ['kompensacijų', 'pašalpų']
Atrinkti kandidatai: ['kompensacijų', 'pašalpų']
Kandidatai įstatymu. : []
Originalus sakinys:  Išmoka gimus vienu metu daugiau kaip vienam vaikui skiri